In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
import re
import time
from scipy import stats

import warnings
warnings.filterwarnings("ignore")
import scipy.stats

#Quality of image:
%config InlineBackend.figure_format = 'svg'
import os

pd.set_option('display.max_columns', 100)

# Import my own functions from utils file:
import import_ipynb
#from utils_home_prices import *

In [2]:
dataDirectory = os.path.join(os.path.dirname(os.getcwd()),'Rental_Cost\\data')
imageDirectory = os.path.join(os.path.dirname(os.getcwd()),'Rental_Cost\\images\\Preparation')

In [3]:
df = pd.read_csv(dataDirectory + "\houses_to_rent.csv")
print(df.shape)
df.head(2)

(10692, 13)


,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa (R$),rent amount (R$),property tax (R$),fire insurance (R$),total (R$)
0,São Paulo,70,2,1,1,7,acept,furnished,2065,3300,211,42,5618
1,São Paulo,320,4,4,0,20,acept,not furnished,1200,4960,1750,63,7973


### Features of the dataset: 
**Unnamed: 0** Incremental id, for prediction puroses we do not need it.
<br>**city:** City where the home is located
<br>**area:** The area of the home,
<br>**rooms:** The count of rooms of the home,
<br>**bathroom:** The count of bathrooms of the home,
<br>**parking spaces:** The count of parking spaces belonged to the home,
<br>**floor:** The floor of the building,
<br>**animal:** Is it animal friendly?
<br>**furniture:** Is it furnished or not?
<br>**hoa** Homeowners association tax,
<br>**property tax:** Monthly property tax,
<br>**property tax:** Homeowners association tax,
<br>**rent amount:** Monthly rental price,
<br>**fire insurance:** Monthly fire insurance payment,
<br>**total** Total value of monthly payment

**1) Renaming columns**

In [4]:
df.rename(columns={'parking spaces': 'parking', 
                   'animal': 'petFriendly',
                   'furniture': 'furnished',
                   'rent amount (R$)': 'rent',
                   'property tax (R$)': 'tax',
                   'fire insurance (R$)': 'fireInsurance',
                   'hoa (R$)': 'hoa',
                   'total (R$)': 'total'
                  },inplace= True)

df.head(2)

,city,area,rooms,bathroom,parking,floor,petFriendly,furnished,hoa,rent,tax,fireInsurance,total
0,São Paulo,70,2,1,1,7,acept,furnished,2065,3300,211,42,5618
1,São Paulo,320,4,4,0,20,acept,not furnished,1200,4960,1750,63,7973


**2) Checking duplicates**

### Data Preparation:
- Checking duplicates,
- Checking missing values,
- Renaming columns,
- Deleting "Unnamed:0" column,
- Modifying city column
- Deleting **"R$ and comma"** in 5 financial columns: (hoa, rent amount, property tax, fire insurance, total),
- Converting string columns into numerical columns,
- Check if there is some non-numerical input within numerical columns,
- Looking for miss values,
- Looking for outliers,
- Checking multi colinearity

In [5]:
duplicated = df.duplicated()
duplicated = duplicated[duplicated.values==True]
duplicated.shape

(358,)

### Insight: 
- There are 358 duplicated rows. 
- However, according to the fact that the data set includes rental homes, it is possible that there are similar records. 
- For example, similar unit in a tall bauilding.
- So, we can keep them.

**3) Checking missing values**

In [6]:
df.isnull().sum()

city             0
area             0
rooms            0
bathroom         0
parking          0
floor            0
petFriendly      0
furnished        0
hoa              0
rent             0
tax              0
fireInsurance    0
total            0
dtype: int64

In [7]:
def make_metadata(df):
    data = []
    for feature in df.columns:
        # Defining the role
        if feature == 'target':
            role = 'target'
        elif feature == 'id':
            role = 'id'
        else:
            role = 'input'

            
        # Defining the data type 
        dtype = df[feature].dtype

        uniqueCnt = df[feature].nunique()
        
        # Defining the type
        if 'bin' in feature or feature == 'target':
            type = 'binary'
        elif 'cat' in feature or feature == 'id':
            type = 'categorical'
        elif (df[feature].dtype == 'float32') | (df[feature].dtype == 'float64'):
            type = 'numeric'
        elif (df[feature].dtype == 'int32') | (df[feature].dtype == 'int64'):
            type = 'ordinal'
        elif (df[feature].dtype == 'object'):
            type = 'categorical'

        if (uniqueCnt == 2 and type == 'ordinal'):
            type = 'binary'


        # Creating a dictionary for adding a row to metadata:
        feature_dictionary = {
            'varname': feature,
            'role': role,
            'type': type,
            'dtype': dtype,
            'uniqueCnt': uniqueCnt
        }
        data.append(feature_dictionary)

    metadata = pd.DataFrame(data, columns=['varname', 'role', 'type', 'dtype', 'uniqueCnt'])
    metadata.set_index('varname', inplace=True)
    return metadata


def make_typedf(df, metadata, typeName):
    cols = metadata[metadata['type']==typeName].index.tolist()
    if 'id' in cols:
        cols.remove('id')
    
    cnt = len(cols)
    
    res_df = pd.DataFrame(columns={'feature', 'uniqueCnt'})
    res_df['feature'] = cols

    for col in cols:
        res_df.loc[res_df['feature']==col,'uniqueCnt'] = df[col].nunique()

    # To set order of columns in dataframe:
    res_df = res_df[['feature', 'uniqueCnt']]
    return cols, cnt, res_df

In [8]:
metadata = make_metadata(df)

catFeatures, catCnt, cat_df = make_typedf(df, metadata, 'categorical')
ordFeatures, ordCnt, ord_df = make_typedf(df, metadata, 'ordinal')
numFeatures, numCnt, num_df = make_typedf(df, metadata, 'numeric')
binFeatures, binCnt, bin_df = make_typedf(df, metadata, 'binary')

metadata

,role,type,dtype,uniqueCnt
varname,,,,
city,input,categorical,object,5
area,input,ordinal,int64,517
rooms,input,ordinal,int64,11
bathroom,input,ordinal,int64,10
parking,input,ordinal,int64,11
floor,input,categorical,object,35
petFriendly,input,categorical,object,2
furnished,input,categorical,object,2
hoa,input,ordinal,int64,1679


In [9]:
def find_nonNumerical_percent(df, col):
    """
    This function checks numerical status of a given column. In fact, it checks if the columns are 100% numerical
    or thay have non-digit as well.
    Input: df: a dataframe, 
           col: a name of a column.
    output: index of non-numerical rows.
    """
    if(df[col].dtype == 'O'):
        print('rexxx')
        check_df = df[col].apply(lambda x: x.isnumeric())
    else:
        print('alii')
        check_df = df[col].apply(lambda x: str(x).isnumeric())
    
    strIndex = check_df.loc[check_df == False].index
    
    if(len(strIndex>0)):
        print('{:0.2f}% of the "{}" column is not numerical.'.format(len(strIndex)*100/df.shape[0], col))
        print('The distribution is as follow:')
        print(df.loc[strIndex, col].value_counts())
        print('********************************************************')
    else:
        print('The "{}" column is verified; 100% numerical'.format(col))
    return strIndex

**5) Preparation of numerical columns:**
- Converting string data into numeric,
- Check if there is some non-numerical data within numerical columns,

In [10]:
def check_numericalStatus(df, cols):
    """
    This function checks if columns passed to the function are numerical or not. 
    If there are some non numerical columns, it will return those columns.
    Input: df: a dataframe, 
           cols: a list of columns for being checked.
    output: a list of non-numerical columns
    """
    
    resType = [df[col].dtype for col in cols]
    res = [col for i,col in enumerate(cols) if resType[i] == 'O']
    if(len(res)>0):
        print('Notice!!!\n')
        print('The following columns are not pure numerical. They have some non-numericla entry.')
        print(res)
    else:
        print('Dataframe is verified!')
        print('All columns are pure numerical.')
    return res

In [11]:
def convert_to_numeric(df, cols, distributionFlag=False):
    """
    This function will convert passed columns into numerical.
    Finally, if there are some columns which are remained in form of object datatype, it will return them.
    Input: df: a dataframe, 
           cols: a list of columns to convert.
           distributionFlag: a flag which indicates to show the distribution of non-numerical columns or not. Default value is False.
    output: df: converted dataframe to numeric (if applicable),
            catCols: list of non-numerical columns,
            res_df: a dataframe including only non-numerical content
            indexLst: a 2d list includes index of non-numerical content
    """
    for col in cols:
        df[col] = pd.to_numeric(df[col], errors='ignore')

    
    resType = [df[col].dtype for col in cols]
    catCols = [col for i,col in enumerate(cols) if resType[i] == 'O']# We will save features having nonnumerical data.
    res_df = pd.DataFrame()
    indexLst = []# We will save index of data which are not numerical in indexLst.


    if(len(catCols)==0):
        print('All columns are verified; 100% numerical')
        return df, catCols, res_df, indexLst
    
    elif(len(catCols)>0): 
        print('Notice!!!\n')
        print('The following columns are not pure numerical. They have some non-numericla data.')
        print(catCols)
        # We keep the info in colLst and rowLst for printing in dataframe.
        colLst = ['Feature', 'NonNumerical %']
        rowLst = []
    
        for col in catCols:
            if(df[col].dtype == 'O'):
                check_status = df[col].apply(lambda x: x.isnumeric())
            else:
                check_status = df[col].apply(lambda x: str(x).isnumeric())
            
            catIndex = check_status.loc[check_status == False].index
            if(len(catIndex>0)):
                indexLst.append(catIndex)
                nonNumericalValue = round(len(catIndex)*100/df.shape[0], 2)
                valueLst = [col, nonNumericalValue]
                dic1 = {k:v for k,v in zip(colLst, valueLst)}
                rowLst.append(dic1)# Will save nonnumerical data for adding into dataframe
                
                if(distributionFlag == True):
                    print('The distribution of nonnumerical data of "{}" is as follow:'.format(col))
                    print(df.loc[catIndex, col].value_counts())
                    print('********************************************************')
        
        res_df = pd.DataFrame(rowLst, columns = colLst)
    return df, catCols, res_df, indexLst

def print_nonNumericalContent(df, nonNumCols, nonNumIndexLst):
    """
    This function will print the content of non-numerical content of the passed columns.
    Input: df: a dataframe, 
           nonNumCols: a list of columns which are non-numerical.
           nonNumIndexLst: is a 2d list which includes the index of non-numerical row.
    output: There is no variable for return. It only print non-numerical content  
    """
    for i, col in enumerate(nonNumCols):
        print('Feature of "{}":'.format(col))
        print(df.loc[nonNumIndexLst[i], col].value_counts())
        print('*******************************************************')
    return

In [12]:
check_numericalStatus(df, ordFeatures)

Dataframe is verified!
All columns are pure numerical.


[]

In [13]:
catFeatures

['city', 'floor', 'petFriendly', 'furnished']

**5) Modifying of Floor Feature:**
- It is considered categorical.
- We have to modify it.

In [14]:
non_num_index = find_nonNumerical_percent(df, 'floor');

rexxx
23.02% of the "floor" column is not numerical.
The distribution is as follow:
-    2461
Name: floor, dtype: int64
********************************************************


In [15]:
df.loc[non_num_index].head(3)

,city,area,rooms,bathroom,parking,floor,petFriendly,furnished,hoa,rent,tax,fireInsurance,total
5,São Paulo,376,3,3,7,-,acept,not furnished,0,8000,834,121,8955
15,Campinas,330,4,6,6,-,acept,furnished,680,8000,328,121,9129
19,São Paulo,600,4,5,6,-,acept,not furnished,0,12000,9500,181,21680


In [16]:
restIndex = ~df.index.isin(non_num_index)
dfTest = df.loc[restIndex]
cityLst = dfTest['city'].unique().tolist()
cityLst

['São Paulo', 'Porto Alegre', 'Rio de Janeiro', 'Campinas', 'Belo Horizonte']

**Imputing floor feature based on the most frequent data:**

In [17]:
cityMode = dfTest.groupby('city')['floor'].agg(pd.Series.mode)
cityMode

city
Belo Horizonte    1
Campinas          1
Porto Alegre      2
Rio de Janeiro    2
São Paulo         1
Name: floor, dtype: object

### Before imputing:

In [18]:
df.loc[df['city']=='São Paulo', 'floor'].value_counts().head()

-    1614
1     542
2     350
3     332
5     307
Name: floor, dtype: int64

### After imputing:

In [19]:
for cityName in cityLst:
    df.loc[(df['city']==cityName) & (df['floor']=='-'), 'floor'] = str(cityMode[cityName])
    
df.loc[df['city']=='São Paulo', 'floor'].value_counts().head()

1    2156
2     350
3     332
5     307
6     299
Name: floor, dtype: int64

In [20]:
df['floor'] = df['floor'].astype(int)

**Updating MetaData:**

In [21]:
metadata = make_metadata(df)

catFeatures, catCnt, cat_df = make_typedf(df, metadata, 'categorical')
ordFeatures, ordCnt, ord_df = make_typedf(df, metadata, 'ordinal')
numFeatures, numCnt, num_df = make_typedf(df, metadata, 'numeric')
binFeatures, binCnt, bin_df = make_typedf(df, metadata, 'binary')

# Let us take a look at cat features:
catFeatures

['city', 'petFriendly', 'furnished']

**Exporting dataset for future using:**

In [22]:
df.to_csv(dataDirectory + "\\rental_houses_Tohid.csv", index=False)